# Практическое задание 7

In [35]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import pandas as pd

In [36]:
response = requests.get('https://lifehacker.ru/kak-vybrat-obogrevatel/')
response.text

'<!doctype html>\n<html data-n-head-ssr lang="ru-RU" data-n-head="%7B%22lang%22:%7B%22ssr%22:%22ru-RU%22%7D%7D">\n  <head >\n    <title>Какой обогреватель выбрать, чтобы точно было тепло — Лайфхакер</title><meta data-n-head="ssr" charset="utf-8"><meta data-n-head="ssr" name="viewport" content="width=device-width, initial-scale=1"><meta data-n-head="ssr" name="format-detection" content="telephone=no,date=no,address=no,email=no,url=no"><meta data-n-head="ssr" data-hid="al:ios:app_name" name="al:ios:app_name" content="Лайфхакер"><meta data-n-head="ssr" name="apple-itunes-app" content="app-id=943373933"><meta data-n-head="ssr" data-hid="al:ios:app_store_id" name="al:ios:app_store_id" content="943373933"><meta data-n-head="ssr" data-hid="og:publisher" property="og:publisher" content="//www.facebook.com/lifehacker.ru"><meta data-n-head="ssr" data-hid="og:site_name" property="og:site_name" content="Лайфхакер"><meta data-n-head="ssr" data-hid="og:type" property="og:type" content="website"><met

##    Определить формат ссылки для пагинации - 1 балл
Смотрим как устроены адреса на нужных страницах и определяем шаблон для пагинации

In [37]:
base_url = 'https://lifehacker.ru/topics/technology/' # базовая часть ссылки, позже в цикле будем добавлять к ней пагинацию
response = requests.get('https://lifehacker.ru/topics/technology/?page=2') # получаем контент первой страиниц
soup = BeautifulSoup(response.text, 'lxml') # инициализируем объект bs4 и задаем парсер lxml

##    В разметке страницы найти уникальные классы/идентификаторы блока с названием и содержанием материала- 3 балла
У нас материалам соответствует класс '''lh-small-article-card__link'''

In [38]:
soup.find_all('a', class_='lh-small-article-card__link') # ищем все a-элементы с классом lh-small-article-card__link

[<a aria-label="10 отличных скидок на гаджеты экосистемы Xiaomi с «Чёрной пятницы» на AliExpress" class="lh-small-article-card__link" data-jest="link" href="/ekosistema-xiaomi-chyornaya-pyatnica-aliexpress/" title="10 отличных скидок на гаджеты экосистемы Xiaomi с «Чёрной пятницы» на AliExpress"></a>,
 <a aria-label="Как получить больше выгоды в М.Видео: тестируем подписку М.Комбо" class="lh-small-article-card__link" data-jest="link" href="/podpiska-m-kombo/" title="Как получить больше выгоды в М.Видео: тестируем подписку М.Комбо"></a>,
 <a aria-label="Зарядки и пауэрбанки Anker с «чёрной пятницы» на AliExpress: выбрали 12 лучших скидок" class="lh-small-article-card__link" data-jest="link" href="/zaryadki-anker-chyornaya-pyatnica-aliexpress/" title="Зарядки и пауэрбанки Anker с «чёрной пятницы» на AliExpress: выбрали 12 лучших скидок"></a>,
 <a aria-label="В Японии разрабатывают «стиральную машину для людей» с искусственным интеллектом" class="lh-small-article-card__link" data-jest="li

##    Получить содержимое десяти страниц списка материалов, выделить из него ссылки на каждый материал - 2 балла

##    Перебрать все полученные ссылки и получить html-код каждого материала - 1 балл

In [39]:
raw_items = soup.find_all('a', class_='lh-small-article-card__link')  # ищем все a-элементы с классом tm-article-comments-counter-link__link
links = [item.get('href') for item in raw_items] # получаем у ссылок только href-атрибут, убираем подстроку '/comments'
links

['/ekosistema-xiaomi-chyornaya-pyatnica-aliexpress/',
 '/podpiska-m-kombo/',
 '/zaryadki-anker-chyornaya-pyatnica-aliexpress/',
 '/stiralnaya-mashina-dlya-lyudei/',
 '/satechi-mac-mini-m4-stand-hub/',
 '/zhivye-foto-honor-300/',
 '/whatsapp-rasshifrovka-golosovyx-soobshhenii/',
 '/razvertyvanie-coloros-15/',
 '/perenos-dannyx-na-android/',
 '/umnye-shtory/',
 '/iphone-se-4-vyjdet-dazhe-ranshe-chem-ozhidalos/',
 '/anons-nubia-z70-ultra/',
 '/obzor-tecno-phantom-v-fold-2/',
 '/anons-ds-pixel-keyboard/',
 '/anons-redmi-a4-5g/',
 '/predstavlen-toshiba-jimucon-sj-9500/',
 '/samsung-raskryla-modeli-chasov-kotorye-poluchat-svezhuyu-wear-os-5/',
 '/predstavleny-casio-g-shock-dw5600/',
 '/top-smartfonov-v-mire/',
 '/kak-vybrat-obogrevatel/',
 '/anons-windows-365-link/',
 '/rezervnaya-kopiya-iphone/',
 '/devaisy-ot-dreame-rasprodazha-chernaya-pyatnitza/',
 '/anons-asus-rog-phone-9/',
 '/otlichnye-umnye-vesy/',
 '/corsair-k65-plus-m75-mac/',
 '/novyi-umnyi-svet-ot-yandeksa/',
 '/luchshie-prilozhe

In [40]:
parsed_urls = []
for page_num in range(1, 10):
    url = f'{base_url}/?page={page_num}' # подставляем нужный номер страницы для пагинации, дальше парсим каждую страницу аналогично коду выше
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'lxml')
    raw_items = soup.find_all('a', class_='lh-small-article-card__link')
    parsed_urls += [item.get('href') for item in raw_items]

##    Распарсить полученный текст разметки с помощью BeautifulSoup, вытащив по каждой ссылке заголовок и содержание материала - 2 баллa
Текст

In [41]:
base_url = 'https://lifehacker.ru' # все русскоязычные материалы имеют ссылки вида https://habr.com/ru/post/718996/

response = requests.get('https://lifehacker.ru/anons-windows-365-link/')
soup = BeautifulSoup(response.text, 'lxml')
soup.find('div', class_='single-article__post-content single-article__content-container').text

'Microsoft анонсировала Windows 365 Link — напоминающий Mac mini миниатюрный ПК, предназначенный для работы с облачным сервисом Windows 365. Устройство представляет собой лёгкий в использовании клиент, позволяющий подключаться к виртуальным компьютерам, размещённым в облаке.Изображение: MicrosoftWindows 365 Link — это бесшумное устройство, которое подключается к локальным мониторам и периферии. Его основная задача — обеспечить максимально простой доступ к ПК, находящимся в облаке. По словам главы Windows и Surface Павана Давулури, компания стремилась сделать устройство максимально простым для пользователя, минимизировав количество шагов от включения до подключения к виртуальной машине.Устройство работает на облегчённой версии Windows, которая запускается за считаные секунды. Локально поддерживается только базовая операционная система, настроенная для быстрого и безопасного подключения к Windows 365. Система является «суперзащищённой» версией Windows: её код изолирован, а все приложения

Заголовок

In [42]:
soup.find('h1', class_='article-card__title').text

'Microsoft представила ПК размером с Mac mini, который работает на облачной версии Windows'

Достаем текст и заголовок каждой статьи

In [43]:
result = []
for url in tqdm(parsed_urls):
    article = {}
    article_url = f'{base_url}{url}' # подставляем часть ссылки, ведущую материал
    response = requests.get(article_url)
    soup = BeautifulSoup(response.text, 'lxml')

    article['title'] = soup.find('h1', class_='article-card__title').text # Получаем текст из span в первом h1 с классом tm-title_h1
    article['text'] = soup.find('div', class_='single-article__post-content single-article__content-container').text # Получаем весь текст из div с классом tm-article-body
   
    result.append(article)


 37%|██████████████▉                         | 101/270 [00:42<02:01,  1.39it/s]


 75%|██████████████████████████████          | 203/270 [01:50<00:41,  1.63it/s]


 98%|███████████████████████████████████████ | 264/270 [02:36<00:04,  1.25it/s]

ConnectionError: HTTPSConnectionPool(host='lifehacker.ruhttps', port=443): Max retries exceeded with url: //lifehacker.ru/special/ora/ (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x000000000B228588>: Failed to establish a new connection: [Errno 11004] getaddrinfo failed'))

##    Создать датайфрейм с полученными данными - 1 балл

In [44]:
pd.set_option('display.max_colwidth', 400)


data = pd.DataFrame(result)
data.head()

,text,title
0,"Компания Casio анонсировала новую модель DW-5000R, которая является обновлённой версией самых первых цифровых часов G-Shock, выпущенных в 1983 году. Новый аксессуар почти полностью повторяет оригинальный дизайн, включая размеры корпуса и детали ремешка, которые были характерны для модели DW-5000C.Изображение: CasioПервые G-Shock придумал инженер Casio Кикуо Ибе, чьи механические часы однажды р...",Casio представила новую версию самых первых цифровых часов G-Shock
1,Оглавление \n Realme GT7 Pro — самый мощный флагман\n \n Nubia Z70 Ultra — самая крутая камера\n \n Xiaomi Redmi A4 — самый народный\n \n Oppo Reno 13 Pro — самый сбалансированный\n \n Nubia Red Magic 10 Pro Plus — самый игровой\n \n Honor X9c — самый защищённый в линейке\n ...,Лучшие смартфоны ноября
2,"Компания Chessnut анонсировала необычное устройство под названием Pixeldarts. Оно представляет собой доску с двумя цветными пиксельными дисплеями и дротиками с сенсорными чипами в комплекте. Играть на ней можно не только в дартс, но и в другие игры — например, в мини-гольф.Изображение: Chessnut Благодаря чипам устройство точно определяет расположение дротиков и то, кто именно их бросает. Можно...",Дартс и не только: представлена пиксельная доска с ретроиграми для компании
3,Оглавление \n Определитесь с типом роутера\n \n Изучите характеристики устройства\n \n Выберите подходящую модель\n Оглавление \n Определитесь с типом роутера\n \n Изучите характеристики устройства\n \n Выберите подходящую модель\n Определитесь с типом роутераИнтернет‑...,"Как выбрать роутер: всё, на чём можно и нельзя сэкономить"
4,"Baseus представила новую модель беспроводных наушников M2s Ultra. Среди главных особенностей — встроенный сенсорный экран на зарядном кейсе и активное шумоподавление до 52 дБ.Изображение: BaseusЗарядный кейс с цветным ЖК-дисплеем позволяет управлять устройством напрямую, без необходимости подключения через приложение. С помощью экрана можно настраивать громкость, переключать режимы воспроизвед...",Представлены TWS-наушники Baseus M2s Ultra с экраном на кейсе и активным шумоподавлением
